In [150]:
import pandas as pd
import html
import string

In [155]:
data_train = pd.read_json('ranking_train.jsonl', lines=True)
new_rows = []
for index, row in data_train.iterrows():
    new_row = {'text': row['text']}
    for i in range(len(row['comments'])):
        new_row['comments_text_' + str(i)] = row['comments'][i]['text']
    new_rows.append(new_row)

pd.DataFrame(new_rows).to_csv('ranking_train.tsv', sep="\t", index=False)

In [ ]:
train_data_tsv = pd.read_csv('ranking_train.tsv', sep="\t")
train_data_tsv[:10]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,There will invariably be those who don't see t...,For me school is a way to be connected to what...,I guess it really depends on how hungry you ar...,I know pollground decided to go back to school...
1,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,Does this mean that there's now a big-name com...,Also on BBC News: http://news.bbc.co.uk/1/low...,I don't understand what they do that is worth ...,sold out too cheaply. given their leadership p...
2,How Costco Became the Anti-Wal-Mart,I really hate it when people falsely claim tha...,"I love Costco. For me, the best quote of the ...","""But Mr. Sinegal warned that if Costco increas...",While Costco does treat its employees a little...,"I'd like to know more about their ""don't adver..."
3,"Fortune Favors Big Turds | Screw The Money, Th...","His real point is that something can be ""simpl...","So I'm thinking, ""Well, I've never heard of th...",I've never seen a single blog post wander in s...,It was a RANT on a personal blog. It's there t...,"In the eternal words of the Geico Caveman..""Wh..."
4,StartupWeekend: 70 Founders Create One Company...,Looks like someone hasn't read The Mythical Ma...,Any chances for recreating this in a telecommu...,"You know, if they started with 70 and eliminat...",And what did you do this weekend?I'd love to h...,Very interesting. I would join an event like t...
5,Younger job seekers hold the advantage (becaus...,I smell bullshit. Note from the article:&#62; ...,"Not a bad read, although I wonder what reality...","More than anything, I think this is about all ...",Some even have their parents in the room for a...,One thing that bugs me about SlapVid is that i...
6,Four Pownce Invites,Ok these are out the door. I have a few more ...,sgoraya at gmail.com\nthx!,kbeshay@gmail.comthanks.,uprz23@gmail.comThanks.,kyle.bolton at gmail
7,LFMs and LFSPs (Michael Vanier via PG),I'd forgotten how well written that was. It's...,"Perfection is achieved, not when there is noth...","To get wider use of LFSPs, it would be importa...",I totally understand people's complaints about...,for (i = 0; i &#60; N; i++)Odd example to use....
8,The Virtues of a Second Screen,I guess I'm one of the few people who has trie...,"I didn't look at the date, but when I saw the ...",I am a big believer in 2 screens as well. I h...,"Bah. A 30"" is vastly superior to two screens. ...",Why is it posted here?
9,"Startup Weekend failed, but lessons learned",70 days and 2 people would work much better th...,"They learned a lesson indeed: ""Development is ...",Is anyone else confused by the premise of star...,&#62; The Java platform was selected/Nuff\n,This is the first I'd heard of startupweekend....


In [153]:
for column in train_data_tsv.columns:
    #заменяем коды на символы
    train_data_tsv[column] = train_data_tsv[column].apply(html.unescape)

    #заменяем url'ы на токен 1url1
    train_data_tsv[column] = train_data_tsv[column].str.replace(
    r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])", 
    '1url1', 
    regex=True)

    #заменяем email'ы на токен 1email1
    train_data_tsv[column] = train_data_tsv[column].str.replace(
        r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*)@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])",
        '1email1', 
        regex=True)
    
    #убираем ссылки
    train_data_tsv[column] = train_data_tsv[column].str.replace(r'\[\d\]', '', regex=True)
    
    # #понижаем регистр
    train_data_tsv[column] = train_data_tsv[column].apply(lambda x: x.lower())

    # #избавляемся от лишних знаков пунктуации
    train_data_tsv[column] = train_data_tsv[column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + "\n")))

    #делаем обратную замену на <token>
    # train_data_tsv[column] = train_data_tsv[column].str.replace('1url1', ' <url> ')
    # train_data_tsv[column] = train_data_tsv[column].str.replace('1email1', ' <email> ')

In [154]:
train_data_tsv.to_csv('ranking_train.tsv', sep="\t", index=False)
train_data_tsv = pd.read_csv('ranking_train.tsv', sep="\t")
train_data_tsv[:10]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,how many summer y combinator fundees decided n...,going back to school is not identical with giv...,there will invariably be those who dont see th...,for me school is a way to be connected to what...,i guess it really depends on how hungry you ar...,i know pollground decided to go back to school...
1,cbs acquires lastfm for 280m,it will be curious to see where this heads in ...,does this mean that theres now a bigname compa...,also on bbc news httpnewsbbccouk1lowtechnolog...,i dont understand what they do that is worth 7...,sold out too cheaply given their leadership po...
2,how costco became the antiwalmart,i really hate it when people falsely claim tha...,i love costco for me the best quote of the ar...,but mr sinegal warned that if costco increased...,while costco does treat its employees a little...,id like to know more about their dont advertis...
3,fortune favors big turds screw the money this...,his real point is that something can be simple...,so im thinking well ive never heard of this bu...,ive never seen a single blog post wander in so...,it was a rant on a personal blog its there to ...,in the eternal words of the geico cavemanwhat
4,startupweekend 70 founders create one company ...,looks like someone hasnt read the mythical man...,any chances for recreating this in a telecommu...,you know if they started with 70 and eliminate...,and what did you do this weekendid love to hea...,very interesting i would join an event like th...
5,younger job seekers hold the advantage because...,i smell bullshit note from the article compani...,not a bad read although i wonder what reality ...,more than anything i think this is about all t...,some even have their parents in the room for a...,one thing that bugs me about slapvid is that i...
6,four pownce invites,ok these are out the door i have a few more h...,sgoraya at gmailcomthx,kbeshaygmailcomthanks,uprz23gmailcomthanks,kylebolton at gmail
7,lfms and lfsps michael vanier via pg,id forgotten how well written that was its be...,perfection is achieved not when there is nothi...,to get wider use of lfsps it would be importan...,i totally understand peoples complaints about ...,for i 0 i n iodd example to use its always a...
8,the virtues of a second screen,i guess im one of the few people who has tried...,i didnt look at the date but when i saw the pr...,i am a big believer in 2 screens as well i ha...,bah a 30 is vastly superior to two screens thi...,why is it posted here
9,startup weekend failed but lessons learned,70 days and 2 people would work much better th...,they learned a lesson indeed development is ha...,is anyone else confused by the premise of star...,the java platform was selectednuff,this is the first id heard of startupweekend t...


In [119]:
data_tf_idf_list = []

for i in range(len(train_data_tsv)): 
    for column in train_data_tsv.columns:
        data_tf_idf_list.append(train_data_tsv[column][i])

pd.DataFrame(data_tf_idf_list).to_csv('ranking_train_tf_idf.tsv', sep="\t", index=False)

In [120]:
train_data_tsv_tf_idf = pd.read_csv('ranking_train_tf_idf.tsv', sep="\t")
train_data_tsv_tf_idf[:10]

,0
0,How many summer Y Combinator fundees decided n...
1,Going back to school is not identical with giv...
2,There will invariably be those who don't see t...
3,For me school is a way to be connected to what...
4,I guess it really depends on how hungry you ar...
5,I know pollground decided to go back to school...
6,CBS acquires last.fm for $280mCBS acquires las...
7,It will be curious to see where this heads in ...
8,Does this mean that there's now a big-name com...
9,Also on BBC News: <url> .Nice to see a Lond...


In [117]:
data_test = pd.read_json('ranking_test.jsonl', lines=True)
new_rows = []
for index, row in data_test.iterrows():
    new_row = {'text': row['text']}
    for i in range(len(row['comments'])):
        new_row['comments_text_' + str(i)] = row['comments'][i]['text']
    new_rows.append(new_row)

pd.DataFrame(new_rows).to_csv('ranking_test.tsv', sep="\t", index=False)

In [84]:
test_data_tsv = pd.read_csv('ranking_test.tsv', sep="\t")
test_data_tsv[:10]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",I&#x27;m still waiting for them to stabilize w...,"For those who upgraded, no need to do a restor...",Upgraded shortly after it was released and suf...,I think they were under a lot of pressure on t...,Fix for those who already updated: http:&#x2F...
1,Ask HN: How do US HNers get their health insur...,Get it from your employer. It&#x27;s better an...,As someone who moved from the UK to US under a...,"I went to http:&#x2F;&#x2F;healthcare.gov, fil...","Full time employed, health insurance through m...",Your options depend on where you live.I signed...
2,San Diego Researcher Crowdfunding Patent-Free ...,What I don&#x27;t understand is why drugs deve...,To everyone sure this will fail because of the...,People don&#x27;t seem to realize how expensiv...,"Many years ago, I worked in a lab where I synt...",I think the negative comments on here have got...
3,Rethinking the origins of the universe,I&#x27;m not a physicist. I imagine that if I...,I&#x27;m not at all acquainted with astrophysi...,This paper claims that black holes can&#x27;t ...,Interesting; if the paper finds general accept...,Ethan Siegel just posted a detailed response t...
4,SlackTextViewController: A new growing text in...,As someone that doesn&#x27;t do iOS developmen...,Apple developed Swift to make developer&#x27;s...,"So I&#x27;m making iOS apps sometimes, but hav...",&gt; Inverted mode for displaying cells upside...,cool. I was just shopping for a new growing t...
5,CVE-2014-6271: Remote code execution through bash,According to http:&#x2F;&#x2F;wiki.bash-hacker...,"Funny, this works even after bash fix &#x2F; u...",It might still be an issue. The patches may no...,Here&#x27;s how to patch Ubuntu 8.04 or anythi...,There&#x27;s some misunderstanding of how the ...
6,"Up, Up, Down, Down, Left, Right, Left, Right, ...",Every time I need to unlock developer settings...,My first thought was that it was a sequence fo...,The contra code . . . you&#x27;ll never forget...,A friend of mine worked on the netflix debuggi...,"Heh, it&#x27;s pretty close to the Konami Code..."
7,Relativistic hash tables,"In case people find it interesting, this is ve...",Could someone explain all of the grounds?,I wrote the paper and dissertation this work w...,"Forgive me if this is stupid, but this could w...","Maybe I&#x27;m daft, but the growing and shrin..."
8,Programming the Atari ST 20 Years Later,I learned basic on a atari 800 i got for chris...,I wrote an Atari ST emulator out of nostalgia ...,An Atari ST was the first computer I ever owne...,&quot; A dark shadow falls over me when I thin...,"Sorry, but the Amiga was wayyyy better."
9,How to Design Classes (2012) [pdf],Is there a mirror of this anywhere? I can&#x27...,666 pages... Coincidence? I don&#x27;t think so.,"For those interested, it looks like programmin...",The first four chapters are written with the P...,Very excited to see how they project their vis...


In [85]:
for column in test_data_tsv.columns:
    #заменяем коды на символы
    test_data_tsv[column] = test_data_tsv[column].apply(html.unescape)

    #заменяем url'ы на токен 1url1
    test_data_tsv[column] = test_data_tsv[column].str.replace(
    r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])", 
    '1url1', 
    regex=True)

    #заменяем email'ы на токен 1email1
    test_data_tsv[column] = test_data_tsv[column].str.replace(
        r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*)@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])",
        '1email1', 
        regex=True)
    
    #убираем ссылки
    test_data_tsv[column] = test_data_tsv[column].str.replace(r'\[\d\]', '', regex=True)
    
    # #понижаем регистр
    # train_data_tsv[column] = train_data_tsv[column].apply(lambda x: x.lower())

    # #избавляемся от лишних знаков пунктуации
    # train_data_tsv[column] = train_data_tsv[column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + "\n")))

    #делаем обратную замену на <token>
    test_data_tsv[column] = test_data_tsv[column].str.replace('1url1', ' <url> ')
    test_data_tsv[column] = test_data_tsv[column].str.replace('1email1', ' <email> ')


In [86]:
test_data_tsv.to_csv('ranking_test.tsv', sep="\t", index=False)
test_data_tsv = pd.read_csv('ranking_test.tsv', sep="\t")
test_data_tsv[:10]

,text,comments_text_0,comments_text_1,comments_text_2,comments_text_3,comments_text_4
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",I'm still waiting for them to stabilize wifi o...,"For those who upgraded, no need to do a restor...",Upgraded shortly after it was released and suf...,I think they were under a lot of pressure on t...,Fix for those who already updated: <url> ...
1,Ask HN: How do US HNers get their health insur...,Get it from your employer. It's better and che...,As someone who moved from the UK to US under a...,"I went to <url> , filled out some online form...","Full time employed, health insurance through m...",Your options depend on where you live.I signed...
2,San Diego Researcher Crowdfunding Patent-Free ...,What I don't understand is why drugs developed...,To everyone sure this will fail because of the...,People don't seem to realize how expensive it ...,"Many years ago, I worked in a lab where I synt...",I think the negative comments on here have got...
3,Rethinking the origins of the universe,I'm not a physicist. I imagine that if I were...,I'm not at all acquainted with astrophysics bu...,This paper claims that black holes can't come ...,Interesting; if the paper finds general accept...,Ethan Siegel just posted a detailed response t...
4,SlackTextViewController: A new growing text in...,As someone that doesn't do iOS development it ...,Apple developed Swift to make developer's more...,"So I'm making iOS apps sometimes, but have not...",> Inverted mode for displaying cells upside-do...,cool. I was just shopping for a new growing t...
5,CVE-2014-6271: Remote code execution through bash,According to <url> it appears that this is b...,"Funny, this works even after bash fix / upgrad...",It might still be an issue. The patches may no...,Here's how to patch Ubuntu 8.04 or anything wh...,There's some misunderstanding of how the one-l...
6,"Up, Up, Down, Down, Left, Right, Left, Right, ...",Every time I need to unlock developer settings...,My first thought was that it was a sequence fo...,The contra code . . . you'll never forget . . ...,A friend of mine worked on the netflix debuggi...,"Heh, it's pretty close to the Konami Code, I w..."
7,Relativistic hash tables,"In case people find it interesting, this is ve...",Could someone explain all of the grounds?,I wrote the paper and dissertation this work w...,"Forgive me if this is stupid, but this could w...","Maybe I'm daft, but the growing and shrinking ..."
8,Programming the Atari ST 20 Years Later,I learned basic on a atari 800 i got for chris...,I wrote an Atari ST emulator out of nostalgia ...,An Atari ST was the first computer I ever owne...,""" A dark shadow falls over me when I think abo...","Sorry, but the Amiga was wayyyy better."
9,How to Design Classes (2012) [pdf],Is there a mirror of this anywhere? I can't ge...,666 pages... Coincidence? I don't think so.,"For those interested, it looks like programmin...",The first four chapters are written with the P...,Very excited to see how they project their vis...


In [ ]:
test_data_tsv_tf_idf = pd.DataFrame({'text' : [''] * len(test_data_tsv)})
for column in test_data_tsv.columns:
    test_data_tsv_tf_idf['text'] += test_data_tsv[column]

In [126]:
from sklearn.metrics import ndcg_score

In [148]:
ndcg_score([[0,1,2,3,4]], [[0,1,2,4,3]], k=5)

0.949604428345699

In [149]:
len(train_data_tsv)

88107

In [ ]:
train_